In [1]:
# -------------------------
# Import modules
# -------------------------

import numpy as np, pandas as pd, matplotlib.pyplot as plt
import itertools, warnings, os
# import sys
# sys.path.insert(0, "/Users/chrismader/Python/DRO")
from dro import *

import warnings
warnings.filterwarnings(
    "ignore", message=r"Argument (sub|subj) .* Incorrect array format causing data to be copied")
pd.set_option('future.no_silent_downcasting', True)
# warnings.filterwarnings("ignore", message="Argument .* in .*: Incorrect array format")

In [11]:
# -------------------------
# CONFIG
# -------------------------

RSLDS_CONFIG = {
    
    # Core defaults
    "dt": 1.0 / 252.0,
    "n_iters": 10,
    "h_z": 10.0,  # CUSUM threshold

    # Batch windows
    "batch_grid": [
        # {"train_window": 504, "overlap_window": 63},
        {"train_window": 756, "overlap_window": 63},
        # {"train_window": 1260, "overlap_window": 63},
    ],

    # Number of regimes
    "K_grid": [3],
    # "K_grid": [2, 3],
    
    # Unrestricted models: 
    "unrestricted_models": [
        # {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        # {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        # {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        # {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        # {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        # {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        # {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        # {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        # {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        # {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        # {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        # {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        # {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        # {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        # {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": False,
    "run_restricted": True,

    # Output
    "display": False,
    "verbose": False,    
}

DRO_CONFIG = {
    
    # General
    "start_dt": None,
    "end_dt": None,
    "min_assets": 3,

    # Risk budget
    "GLOBAL": {"risk_budget": 0.30, "risk_free_rate": 0.0, "epsilon_sigma": 1e-6,},
    
    # Ambiguity set
    "delta_name": "kappa_l2",

    # Data
    "use_gridsearch_csv": False,
    "gridsearch_results_csv":  "/content/drive/MyDrive/Colab Notebooks/SLDS/Output/gridsearch_results_test.csv",
    "gridsearch_segments_csv": "/content/drive/MyDrive/Colab Notebooks/SLDS/Output/gridsearch_segments_test.csv",
}


DELTA_DEFAULTS = {
    "kappa_l2": {"delta_method": "kappa_l2", "kappa": 1.0},
    "kappa_rate": {"delta_method": "kappa_rate", "kappa": 1.0},
    "bound_ek": {"delta_method": "bound_ek", "alpha": 0.05, "c1": 3.0, "c2": 1.0, "a": 2.0},
    "bootstrap_np": {"delta_method": "bootstrap_np", "alpha": 0.05, "B": 100, "seed": 0},
    "bootstrap_gaussian": {"delta_method": "bootstrap_gaussian", "alpha": 0.05, "B": 100, "seed": 0},
}


In [13]:
# -------------------------
# Execute
# -------------------------

if __name__ == "__main__":
    
    filename = "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/bbg_data.xlsx"
    out_path = "/content/drive/MyDrive/Colab Notebooks/SLDS/Output"
    
    # df = pd.read_excel(filename, sheet_name="SPX_PX", skiprows=4, index_col=0)
    # df = df.iloc[3:, :]
    # securities = df.columns.to_list()
    
    securities = ['MSFT', 'NVDA', 'AAPL']  # , 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA']

    # securities = ["BAC", "ORCL", "IBM", "CSCO", "AMD", "ADBE", "GOOG", "T", "HD", "MCD", "PG", "WFC", "SPGI", "SCHW", "C"]

    '''
    securities = [
        "MSFT","NVDA","AAPL","AVGO","GOOGL","ORCL","IBM","CSCO","CRM","AMD","INTU","NOW","TXN","QCOM",
        "ADBE","AMAT","PLTR","ACN","META","NFLX","GOOGL","GOOG","DIS","VZ","T","UBER","AMZN","TSLA","HD",
        "MCD","BKNG","COST","WMT","PG","KO","PM","PEP","BRK/B","JPM","V","MA","BAC","WFC","GS","MS","SPGI",
        "SCHW","AXP","BLK","C","PGR","LLY","JNJ","ABBV","UNH","ABT","MRK","ISRG","BSX","TMO","AMGN","RTX",
        "XOM","CVX","GE","CAT","BA","HON","LIN","NEE","DUK","SO","AEP","D","AMT","PLD","EQIX","O","CBRE"]
    '''
    
    file = "/content/drive/MyDrive/Colab Notebooks/SLDS/Output/gridsearch_results_test.csv"
    df = pd.read_csv(file)
    securities = df["security"].unique()
    
    out = dro_pipeline(securities, filename, out_path, RSLDS_CONFIG, DRO_CONFIG, DELTA_DEFAULTS)


[DEBUG:MODE] use_gridsearch_csv=False
[DEBUG:R] R_use.shape=(3521, 3)  R_use_clean.shape=(3521, 3)
[DEBUG:R] cols=['MSFT', 'NVDA', 'AMD']
[DEBUG:R] index span: 2012-01-02 00:00:00 → 2025-06-30 00:00:00  (len=3521)
[DEBUG:DATA_A] train.shape=(3521, 3)  test.shape=(3521, 3)
[DEBUG:DATA_A] columns(train)==columns(test)? True
[DEBUG:DATA_A] columns=['MSFT', 'NVDA', 'AMD']
[DEBUG:DATA_A] NaNs(train) per col={'MSFT': 0, 'NVDA': 0, 'AMD': 0}
[DEBUG:DATA_A] NaNs(test)  per col={'MSFT': 0, 'NVDA': 0, 'AMD': 0}
[DEBUG:DATA_A] train index: 2012-01-02 00:00:00 → 2025-06-30 00:00:00 (len=3521)
[DEBUG:DATA_A] test  index:  2012-01-02 00:00:00 → 2025-06-30 00:00:00 (len=3521)
[DEBUG:DATA_A] dtypes(train)=['float64', 'float64', 'float64']
[DEBUG:DATA_A] dtypes(test) =['float64', 'float64', 'float64']
[DEBUG:DATA_A] sha(train)=8d40ae48611b13ca  sha(test)=8d40ae48611b13ca
[DEBUG:FIT_A] N=3 (assets in DATA_A)


TypeError: object of type 'NoneType' has no len()

In [ ]:
# summaries (guaranteed to show)
summA = pd.Series(out["PartA"]["summary"]).round(4)
summB = pd.Series(out["PartB"]["summary"]).round(4)
print("\n[Part A] Static DRO summary:\n", summA); sys.stdout.flush()
print("\n[Part B] Regime-DRO summary:\n", summB); sys.stdout.flush()

# portfolios (weights)
print("\n[Part A] weights:\n", out["PartA"]["fit"]["w"]); sys.stdout.flush()
print("\n[Part B] piecewise weights:")
for k, w in enumerate(out["PartB"]["fit"]["w_list"], 1):
    print(f"  k={k}: {w}"); sys.stdout.flush()


In [ ]:
# avg holdings period
rebal=[0, 8, 24]
avg_holding_period = max(rebal)/(len(rebal)-1)
avg_holding_period